In [5]:
# Delete old incompatible checkpoints and start fresh
import shutil
import os

checkpoint_dir = '/content/checkpoints'
results_dir = '/content/results'

if os.path.exists(checkpoint_dir):
    print("Removing old incompatible checkpoints...")
    shutil.rmtree(checkpoint_dir)

if os.path.exists(results_dir):
    print("Removing old resutls...")
    shutil.rmtree(results_dir)

Removing old incompatible checkpoints...
Removing old resutls...


In [6]:
mkdir checkpoints results

In [3]:
ls

checkpoints/  results/  sample_data/


In [ ]:
"""
VQ-VAE Emoji Generation - Phase 1: Codebook Training
Focus on achieving good codebook utilization (50-60%) before proceeding
"""
import os
import json
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.utils import make_grid
from skimage.metrics import structural_similarity as ssim
import warnings
warnings.filterwarnings('ignore')



# ============================================================================
# Configuration Management
# ============================================================================

# Define all configurations in JSON format
EXPERIMENT_CONFIGS = {
    # Paths
    "data_dir": "./emoji_data",
    "checkpoint_dir": "./checkpoints",
    "results_dir": "./results",

    # Data
    "image_size": 64,
    "batch_size": 64,
    "num_workers": 2,

    # VQ-VAE Architecture
    "num_hiddens": 128,
    "num_residual_hiddens": 32,
    "num_residual_layers": 2,
    "embedding_dim": 64,
    "num_embeddings": 256,
    "commitment_cost": 0.01,
    "decay": 0.95,

    # Training
    "num_epochs_vqvae": 100,
    "learning_rate_vqvae": 3e-4,

    # Codebook monitoring
    "min_codebook_usage": 50.0,
    "check_usage_every": 5,

    # Experiment metadata
    "experiment_name": "commitment_cost_10",
    "notes": "Decreasing commitment cost to 10"
}


# ============================================================================
# Configuration
# ============================================================================

class Config:
    """Configuration class that loads from JSON dict"""
    def __init__(self, config_dict):
        for key, value in config_dict.items():
            setattr(self, key, value)

        # Add device (not in JSON as it's system-dependent)
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'

    def to_dict(self):
        """Convert config back to dictionary for CSV export"""
        return {k: v for k, v in self.__dict__.items() if not k.startswith('_')}

config = Config(EXPERIMENT_CONFIGS)


# ============================================================================
# Dataset
# ============================================================================

class EmojiDataset(Dataset):
    def __init__(self, data_dir, image_size=64, transform=None):
        self.data_dir = data_dir
        self.image_size = image_size
        self.image_files = [f for f in os.listdir(data_dir) if f.endswith(('.png', '.jpg', '.jpeg'))]

        if transform is None:
            self.transform = transforms.Compose([
                transforms.Resize((image_size, image_size)),
                transforms.ToTensor(),
                transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
            ])
        else:
            self.transform = transform

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = os.path.join(self.data_dir, self.image_files[idx])
        image = Image.open(img_path).convert('RGB')
        return self.transform(image)

def download_emoji_dataset():
    print("Downloading emoji dataset...")
    try:
        from datasets import load_dataset
        dataset = load_dataset("valhalla/emoji-dataset", split="train")

        print(f"Downloaded {len(dataset)} emojis")
        print("Saving images to disk...")

        for idx, item in enumerate(tqdm(dataset)):
            img = item['image']
            if img.mode != 'RGB':
                img = img.convert('RGB')
            img.save(os.path.join(config.data_dir, f'emoji_{idx:05d}.png'))

        print(f"Saved {len(dataset)} emoji images to {config.data_dir}")
    except Exception as e:
        print(f"Error downloading dataset: {e}")

# ============================================================================
# VQ-VAE Components
# ============================================================================

class VectorQuantizerEMA(nn.Module):
    def __init__(self, num_embeddings, embedding_dim, commitment_cost, decay=0.99, epsilon=1e-5):
        super().__init__()
        self.embedding_dim = embedding_dim
        self.num_embeddings = num_embeddings
        self.commitment_cost = commitment_cost
        self.decay = decay
        self.epsilon = epsilon

        embed = torch.randn(num_embeddings, embedding_dim)
        self.register_buffer("embed", embed)
        self.register_buffer("cluster_size", torch.zeros(num_embeddings))
        self.register_buffer("embed_avg", embed.clone())

    def forward(self, inputs):
        input_shape = inputs.shape
        flat_input = inputs.permute(0, 2, 3, 1).contiguous().view(-1, self.embedding_dim)

        distances = (torch.sum(flat_input**2, dim=1, keepdim=True)
                    + torch.sum(self.embed**2, dim=1)
                    - 2 * torch.matmul(flat_input, self.embed.t()))

        encoding_indices = torch.argmin(distances, dim=1).unsqueeze(1)
        encodings = torch.zeros(encoding_indices.shape[0], self.num_embeddings, device=inputs.device)
        encodings.scatter_(1, encoding_indices, 1)

        quantized = torch.matmul(encodings, self.embed)

        if self.training:
            self.cluster_size.data.mul_(self.decay).add_(encodings.sum(0), alpha=1 - self.decay)
            dw = torch.matmul(encodings.t(), flat_input)
            self.embed_avg.data.mul_(self.decay).add_(dw, alpha=1 - self.decay)

            n = self.cluster_size.sum()
            cluster_size = ((self.cluster_size + self.epsilon) / (n + self.num_embeddings * self.epsilon) * n)
            embed_normalized = self.embed_avg / cluster_size.unsqueeze(1)
            self.embed.data.copy_(embed_normalized)

        e_latent_loss = F.mse_loss(quantized.detach(), flat_input)
        loss = self.commitment_cost * e_latent_loss
        quantized = flat_input + (quantized - flat_input).detach()

        quantized = quantized.view(input_shape[0], input_shape[2], input_shape[3], self.embedding_dim)
        quantized = quantized.permute(0, 3, 1, 2).contiguous()

        avg_probs = torch.mean(encodings, dim=0)
        perplexity = torch.exp(-torch.sum(avg_probs * torch.log(avg_probs + 1e-10)))

        return quantized, loss, perplexity, encoding_indices

class ResidualBlock(nn.Module):
    def __init__(self, in_channels, num_hiddens, num_residual_hiddens):
        super().__init__()
        self.block = nn.Sequential(
            nn.ReLU(),
            nn.Conv2d(in_channels, num_residual_hiddens, 3, padding=1, bias=False),
            nn.BatchNorm2d(num_residual_hiddens),
            nn.ReLU(),
            nn.Conv2d(num_residual_hiddens, num_hiddens, 1, bias=False),
            nn.BatchNorm2d(num_hiddens)
        )

    def forward(self, x):
        return x + self.block(x)

class ResidualStack(nn.Module):
    def __init__(self, in_channels, num_hiddens, num_residual_layers, num_residual_hiddens):
        super().__init__()
        self.layers = nn.ModuleList([
            ResidualBlock(in_channels, num_hiddens, num_residual_hiddens)
            for _ in range(num_residual_layers)
        ])

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return F.relu(x)

class Encoder(nn.Module):
    def __init__(self, in_channels, num_hiddens, num_residual_layers, num_residual_hiddens):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels, num_hiddens // 2, 4, stride=2, padding=1)
        self.conv2 = nn.Conv2d(num_hiddens // 2, num_hiddens, 4, stride=2, padding=1)
        self.conv3 = nn.Conv2d(num_hiddens, num_hiddens, 3, padding=1)
        self.residual_stack = ResidualStack(num_hiddens, num_hiddens, num_residual_layers, num_residual_hiddens)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.conv3(x)
        return self.residual_stack(x)

class Decoder(nn.Module):
    def __init__(self, embedding_dim, num_hiddens, num_residual_layers, num_residual_hiddens):
        super().__init__()
        self.conv1 = nn.Conv2d(embedding_dim, num_hiddens, 3, padding=1)
        self.residual_stack = ResidualStack(num_hiddens, num_hiddens, num_residual_layers, num_residual_hiddens)
        self.conv_trans1 = nn.ConvTranspose2d(num_hiddens, num_hiddens // 2, 4, stride=2, padding=1)
        self.conv_trans2 = nn.ConvTranspose2d(num_hiddens // 2, 3, 4, stride=2, padding=1)

    def forward(self, x):
        x = self.conv1(x)
        x = self.residual_stack(x)
        x = F.relu(self.conv_trans1(x))
        return torch.tanh(self.conv_trans2(x))

class VQVAE(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.encoder = Encoder(3, config.num_hiddens, config.num_residual_layers, config.num_residual_hiddens)
        self.pre_vq_conv = nn.Conv2d(config.num_hiddens, config.embedding_dim, 1)
        self.vq = VectorQuantizerEMA(config.num_embeddings, config.embedding_dim, config.commitment_cost, decay=config.decay)
        self.decoder = Decoder(config.embedding_dim, config.num_hiddens, config.num_residual_layers, config.num_residual_hiddens)

    def forward(self, x):
        z = self.encoder(x)
        z = self.pre_vq_conv(z)
        quantized, vq_loss, perplexity, encoding_indices = self.vq(z)
        x_recon = self.decoder(quantized)
        return x_recon, vq_loss, perplexity, encoding_indices

    def encode(self, x):
        z = self.encoder(x)
        z = self.pre_vq_conv(z)
        _, _, _, encoding_indices = self.vq(z)
        B = x.shape[0]
        return encoding_indices.view(B, -1)

# ============================================================================
# Metrics
# ============================================================================

def calculate_mse(original, reconstructed):
    return F.mse_loss(reconstructed, original).item()

def calculate_psnr(original, reconstructed, max_val=2.0):
    mse = F.mse_loss(reconstructed, original)
    return (10 * torch.log10(max_val**2 / mse)).item()

def calculate_ssim(original, reconstructed):
    orig_np = (original.detach().cpu().numpy() + 1) / 2
    recon_np = (reconstructed.detach().cpu().numpy() + 1) / 2
    ssim_scores = [ssim(orig_np[i].transpose(1, 2, 0), recon_np[i].transpose(1, 2, 0),
                       channel_axis=2, data_range=1.0) for i in range(orig_np.shape[0])]
    return np.mean(ssim_scores)

def calculate_codebook_usage(model, dataloader, device):
    """Calculate codebook utilization percentage"""
    model.eval()
    all_codes = []

    with torch.no_grad():
        for batch in dataloader:
            batch = batch.to(device)
            codes = model.encode(batch)
            all_codes.append(codes.cpu())

    all_codes = torch.cat(all_codes, dim=0).numpy().flatten()
    unique_codes = len(np.unique(all_codes))
    usage_percentage = (unique_codes / config.num_embeddings) * 100

    return usage_percentage, unique_codes, all_codes

# ============================================================================
# Trainer with Codebook Monitoring
# ============================================================================

class Trainer:
    def __init__(self, model, config):
        self.model = model
        self.config = config
        self.optimizer = torch.optim.Adam(model.parameters(), lr=config.learning_rate_vqvae)
        self.history = {'recon_loss': [], 'vq_loss': [], 'total_loss': [], 'perplexity': [],
                       'mse': [], 'psnr': [], 'ssim': [], 'codebook_usage': []}
        self.start_epoch = 0
        self.best_usage = 0

    def save_checkpoint(self, epoch, filepath):
        checkpoint = {
            'epoch': epoch,
            'model_state_dict': self.model.state_dict(),
            'optimizer_state_dict': self.optimizer.state_dict(),
            'history': self.history
        }
        torch.save(checkpoint, filepath)
        print(f"✓ Checkpoint saved: {filepath}")

    def load_checkpoint(self, filepath):
        if os.path.exists(filepath):
            try:
                checkpoint = torch.load(filepath, map_location=self.config.device)
                self.model.load_state_dict(checkpoint['model_state_dict'])
                self.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
                self.history = checkpoint['history']
                self.start_epoch = checkpoint['epoch'] + 1
                print(f"✓ Checkpoint loaded, resuming from epoch {self.start_epoch}")
                return True
            except Exception as e:
                print(f"✗ Error loading checkpoint: {e}")
                return False
        return False

    def train_epoch(self, dataloader):
        self.model.train()
        epoch_recon_loss = epoch_vq_loss = epoch_perplexity = 0
        pbar = tqdm(dataloader, desc="Training")
        for batch in pbar:
            batch = batch.to(self.config.device)
            recon, vq_loss, perplexity, _ = self.model(batch)
            recon_loss = F.mse_loss(recon, batch)
            loss = recon_loss + vq_loss
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
            epoch_recon_loss += recon_loss.item()
            epoch_vq_loss += vq_loss.item()
            epoch_perplexity += perplexity.item()
            pbar.set_postfix({'recon_loss': recon_loss.item(), 'vq_loss': vq_loss.item(), 'perplexity': perplexity.item()})
        n = len(dataloader)
        return {'recon_loss': epoch_recon_loss/n, 'vq_loss': epoch_vq_loss/n, 'perplexity': epoch_perplexity/n}

    @torch.no_grad()
    def evaluate(self, dataloader):
        self.model.eval()
        all_original, all_recon = [], []
        for batch in dataloader:
            batch = batch.to(self.config.device)
            recon, _, _, _ = self.model(batch)
            all_original.append(batch)
            all_recon.append(recon)
        original = torch.cat(all_original, dim=0)
        reconstructed = torch.cat(all_recon, dim=0)
        return {'mse': calculate_mse(original, reconstructed),
                'psnr': calculate_psnr(original, reconstructed),
                'ssim': calculate_ssim(original[:64], reconstructed[:64])}

    def train(self, train_loader, val_loader):
        print(f"\n{'='*80}")
        print(f"PHASE 1: VQ-VAE TRAINING WITH CODEBOOK MONITORING")
        print(f"Target: ≥{self.config.min_codebook_usage}% codebook utilization")
        print(f"{'='*80}\n")

        for epoch in range(self.start_epoch, self.config.num_epochs_vqvae):
            print(f"\nEpoch {epoch+1}/{self.config.num_epochs_vqvae}")

            train_metrics = self.train_epoch(train_loader)
            val_metrics = self.evaluate(val_loader)

            # Check codebook usage periodically
            if (epoch + 1) % self.config.check_usage_every == 0:
                usage_pct, unique_codes, _ = calculate_codebook_usage(self.model, train_loader, self.config.device)
                self.history['codebook_usage'].append(usage_pct)

                print(f"\n{'─'*60}")
                print(f"📊 CODEBOOK USAGE CHECK (Epoch {epoch+1})")
                print(f"{'─'*60}")
                print(f"Active codes: {unique_codes}/{self.config.num_embeddings}")
                print(f"Usage: {usage_pct:.2f}%")

                if usage_pct > self.best_usage:
                    self.best_usage = usage_pct
                    print(f"✓ New best usage: {usage_pct:.2f}%")
                    self.save_checkpoint(epoch, os.path.join(self.config.checkpoint_dir, 'vqvae_best_usage.pt'))

                if usage_pct >= self.config.min_codebook_usage:
                    print(f"✓ TARGET REACHED! ({usage_pct:.2f}% ≥ {self.config.min_codebook_usage}%)")
                else:
                    remaining = self.config.min_codebook_usage - usage_pct
                    print(f"⚠ Need {remaining:.2f}% more to reach target")
                print(f"{'─'*60}\n")

            metrics = {**train_metrics, **val_metrics}
            for key, value in metrics.items():
                self.history[key].append(value)

            print(f"Recon: {metrics['recon_loss']:.4f}, VQ: {metrics['vq_loss']:.4f}, Perplexity: {metrics['perplexity']:.2f}")
            print(f"MSE: {metrics['mse']:.4f}, PSNR: {metrics['psnr']:.2f}, SSIM: {metrics['ssim']:.4f}")

            if (epoch + 1) % 5 == 0:
                self.save_checkpoint(epoch, os.path.join(self.config.checkpoint_dir, f'vqvae_epoch_{epoch+1}.pt'))

        self.save_checkpoint(self.config.num_epochs_vqvae - 1, os.path.join(self.config.checkpoint_dir, 'vqvae_final.pt'))

        # Final usage check
        final_usage, final_unique, _ = calculate_codebook_usage(self.model, train_loader, self.config.device)
        print(f"\n{'='*80}")
        print(f"PHASE 1 COMPLETE")
        print(f"{'='*80}")
        print(f"Final codebook usage: {final_usage:.2f}% ({final_unique}/{self.config.num_embeddings})")
        print(f"Best codebook usage: {self.best_usage:.2f}%")

        if final_usage >= self.config.min_codebook_usage:
            print(f"✓ SUCCESS: Codebook utilization target achieved!")
            return True, self.history
        else:
            print(f"⚠ WARNING: Codebook utilization below target ({final_usage:.2f}% < {self.config.min_codebook_usage}%)")
            print(f"Consider: 1) Training longer, 2) Reducing num_embeddings, 3) Adjusting commitment_cost")
            return False, self.history

# ============================================================================
# Visualization
# ============================================================================

def plot_training_history(history, save_path):
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    axes[0, 0].plot(history['recon_loss'])
    axes[0, 0].set_title('Reconstruction Loss')
    axes[0, 1].plot(history['vq_loss'])
    axes[0, 1].set_title('VQ Loss')
    axes[0, 2].plot(history['perplexity'])
    axes[0, 2].set_title('Perplexity')
    axes[1, 0].plot(history['mse'])
    axes[1, 0].set_title('MSE')
    axes[1, 1].plot(history['psnr'])
    axes[1, 1].set_title('PSNR')
    axes[1, 2].plot(history['ssim'])
    axes[1, 2].set_title('SSIM')
    for ax in axes.flat:
        ax.grid(True)
        ax.set_xlabel('Epoch')
    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()

def plot_reconstructions(model, dataloader, device, save_path, num_images=8):
    model.eval()
    images = next(iter(dataloader))[:num_images].to(device)
    with torch.no_grad():
        reconstructions, _, _, _ = model(images)
    images, reconstructions = (images + 1) / 2, (reconstructions + 1) / 2
    comparison = torch.cat([images, reconstructions])
    grid = make_grid(comparison, nrow=num_images, padding=2)
    plt.figure(figsize=(16, 4))
    plt.imshow(grid.permute(1, 2, 0).cpu().numpy())
    plt.axis('off')
    plt.title('Top: Original, Bottom: Reconstructed')
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()

def plot_codebook_usage(history, all_codes, save_path):
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))

    # Usage over time
    if len(history['codebook_usage']) > 0:
        epochs = [i * config.check_usage_every for i in range(1, len(history['codebook_usage']) + 1)]
        axes[0].plot(epochs, history['codebook_usage'], marker='o', linewidth=2)
        axes[0].axhline(y=config.min_codebook_usage, color='r', linestyle='--', label=f'Target ({config.min_codebook_usage}%)')
        axes[0].set_xlabel('Epoch')
        axes[0].set_ylabel('Codebook Usage (%)')
        axes[0].set_title('Codebook Utilization Over Time')
        axes[0].legend()
        axes[0].grid(True, alpha=0.3)

    # Histogram
    axes[1].hist(all_codes, bins=config.num_embeddings, edgecolor='black')
    axes[1].set_xlabel('Code Index')
    axes[1].set_ylabel('Frequency')
    axes[1].set_title(f'Code Distribution (Final)')
    axes[1].grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()

def save_results_to_csv(config, results, save_path):
    """Save experiment config and results to CSV"""
    import csv
    from datetime import datetime

    # Combine config and results
    row_data = {
        'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        **config.to_dict(),
        **results
    }

    # Check if file exists to determine if we need headers
    file_exists = os.path.exists(save_path)

    with open(save_path, 'a', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=row_data.keys())
        if not file_exists:
            writer.writeheader()
        writer.writerow(row_data)

    print(f"Results appended to: {save_path}")

# ============================================================================
# Main
# ============================================================================

def main():
    print("="*80)
    print("VQ-VAE EMOJI GENERATION - PHASE 1: CODEBOOK TRAINING")
    print("="*80)

    # Dataset
    if not os.listdir(config.data_dir):
        download_emoji_dataset()

    dataset = EmojiDataset(config.data_dir, config.image_size)
    train_size = int(0.9 * len(dataset))
    val_size = len(dataset) - train_size
    train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])
    train_loader = DataLoader(train_dataset, batch_size=config.batch_size, shuffle=True, num_workers=config.num_workers, pin_memory=True)
    val_loader = DataLoader(val_dataset, batch_size=config.batch_size, shuffle=False, num_workers=config.num_workers, pin_memory=True)

    print(f"Train: {len(train_dataset)}, Val: {len(val_dataset)}")

    # Train VQ-VAE
    vqvae = VQVAE(config).to(config.device)
    trainer = Trainer(vqvae, config)

    if not trainer.load_checkpoint(os.path.join(config.checkpoint_dir, 'vqvae_final.pt')):
        success, history = trainer.train(train_loader, val_loader)
        plot_training_history(history, os.path.join(config.results_dir, 'training_history.png'))

        # Final codebook analysis
        usage_pct, unique_codes, all_codes = calculate_codebook_usage(vqvae, train_loader, config.device)
        plot_codebook_usage(history, all_codes, os.path.join(config.results_dir, 'codebook_usage.png'))

        if not success:
            print("\n⚠ Phase 1 did not reach target codebook usage. Review suggestions above.")
            return

    # Visualize results
    plot_reconstructions(vqvae, val_loader, config.device, os.path.join(config.results_dir, 'reconstructions.png'))

    # Save summary
    usage_pct, unique_codes, _ = calculate_codebook_usage(vqvae, train_loader, config.device)
    summary = {
        'codebook_usage_percent': float(usage_pct),
        'active_codes': int(unique_codes),
        'total_codes': config.num_embeddings,
        'final_perplexity': trainer.history['perplexity'][-1] if trainer.history['perplexity'] else None,
        'target_achieved': usage_pct >= config.min_codebook_usage
    }

    with open(os.path.join(config.results_dir, 'phase1_summary.json'), 'w') as f:
        json.dump(summary, f, indent=2)


    # Prepare results for CSV
    results = {
        'phase': 'phase1',
        'codebook_usage_percent': float(usage_pct),
        'active_codes': int(unique_codes),
        'best_codebook_usage': trainer.best_usage,
        'final_perplexity': trainer.history['perplexity'][-1] if trainer.history['perplexity'] else None,
        'final_recon_loss': trainer.history['recon_loss'][-1] if trainer.history['recon_loss'] else None,
        'final_vq_loss': trainer.history['vq_loss'][-1] if trainer.history['vq_loss'] else None,
        'final_mse': trainer.history['mse'][-1] if trainer.history['mse'] else None,
        'final_psnr': trainer.history['psnr'][-1] if trainer.history['psnr'] else None,
        'final_ssim': trainer.history['ssim'][-1] if trainer.history['ssim'] else None,
        'target_achieved': usage_pct >= config.min_codebook_usage,
        'training_completed': True
    }

    # Save to CSV
    csv_path = os.path.join(config.results_dir, 'experiment_results.csv')
    save_results_to_csv(config, results, csv_path)

    # Also save JSON for Phase 2 compatibility
    with open(os.path.join(config.results_dir, 'phase1_summary.json'), 'w') as f:
        json.dump(summary, f, indent=2)

    print(f"\n✓ Phase 1 complete. Results saved to {config.results_dir}")
    print(f"✓ Ready for Phase 2: Prior training and generation")

if __name__ == "__main__":
    main()

VQ-VAE EMOJI GENERATION - PHASE 1: CODEBOOK TRAINING


FileNotFoundError: [Errno 2] No such file or directory: './emoji_data'

In [ ]:
# @title
"""
VQ-VAE Emoji Generation Project - ENHANCED VERSION
Complete implementation with improved codebook utilization monitoring and all analysis from both codes
"""
import os
import json
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.utils import make_grid
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
import seaborn as sns
from scipy import linalg
from skimage.metrics import structural_similarity as ssim
import warnings
warnings.filterwarnings('ignore')

# ============================================================================
# Configuration
# ============================================================================
class Config:
    # Paths
    data_dir = './emoji_data'
    checkpoint_dir = './checkpoints'
    results_dir = './results'

    # Data
    image_size = 64
    batch_size = 64  # Reduced from 128 for stability
    num_workers = 2

    # VQ-VAE Architecture - UPDATED FOR BETTER CODEBOOK USAGE
    num_hiddens = 128
    num_residual_hiddens = 32
    num_residual_layers = 2
    embedding_dim = 64
    num_embeddings = 512
    commitment_cost = 0.25

    # Add EMA for codebook (optional but helps)
    decay = 0.999

    # Training
    num_epochs_vqvae = 100
    learning_rate_vqvae = 3e-4
    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    # Prior (PixelCNN) - UPDATED
    num_epochs_prior = 50
    learning_rate_prior = 1e-4  # Reduced from 3e-4 for stability
    pixelcnn_layers = 12  # Reduced from 15
    pixelcnn_hidden = 64  # Reduced from 128
    grad_clip = 1.0  # Add gradient clipping

    # Generation
    num_samples = 64

    # Interpolation
    num_interpolation_steps = 10

    # Codebook utilization threshold
    codebook_utilization_threshold = 0.5  # 50% minimum utilization

config = Config()

# Create directories
os.makedirs(config.checkpoint_dir, exist_ok=True)
os.makedirs(config.results_dir, exist_ok=True)
os.makedirs(config.data_dir, exist_ok=True)
print(f"Using device: {config.device}")

# ============================================================================
# Dataset Preparation
# ============================================================================
class EmojiDataset(Dataset):
    """Dataset class for emoji images"""

    def __init__(self, data_dir, image_size=64, transform=None):
        self.data_dir = data_dir
        self.image_size = image_size
        self.image_files = [f for f in os.listdir(data_dir)
                           if f.endswith(('.png', '.jpg', '.jpeg'))]

        if transform is None:
            self.transform = transforms.Compose([
                transforms.Resize((image_size, image_size)),
                transforms.ToTensor(),
                transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
            ])
        else:
            self.transform = transform

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = os.path.join(self.data_dir, self.image_files[idx])
        image = Image.open(img_path).convert('RGB')
        image = self.transform(image)
        return image

def download_emoji_dataset():
    """Download emoji dataset from HuggingFace"""
    print("Downloading emoji dataset...")
    try:
        from datasets import load_dataset
        dataset = load_dataset("valhalla/emoji-dataset", split="train")

        print(f"Downloaded {len(dataset)} emojis")
        print("Saving images to disk...")

        for idx, item in enumerate(tqdm(dataset)):
            img = item['image']
            if img.mode != 'RGB':
                img = img.convert('RGB')
            img.save(os.path.join(config.data_dir, f'emoji{idx:05d}.png'))

        print(f"Saved {len(dataset)} emoji images to {config.data_dir}")
    except Exception as e:
        print(f"Error downloading dataset: {e}")
        print("Please manually download emojis to the data directory")

# ============================================================================
# VQ-VAE Model Components
# ============================================================================
class VectorQuantizerEMA(nn.Module):
    """Vector Quantization with Exponential Moving Average"""

    def __init__(self, num_embeddings, embedding_dim, commitment_cost, decay=0.99, epsilon=1e-5):
        super().__init__()
        self.embedding_dim = embedding_dim
        self.num_embeddings = num_embeddings
        self.commitment_cost = commitment_cost
        self.decay = decay
        self.epsilon = epsilon

        # Initialize embeddings correctly - shape should be [num_embeddings, embedding_dim]
        embed = torch.randn(num_embeddings, embedding_dim)
        self.register_buffer("embed", embed)
        self.register_buffer("cluster_size", torch.zeros(num_embeddings))
        self.register_buffer("embed_avg", embed.clone())

    def forward(self, inputs):
        # inputs: [B, C, H, W]
        input_shape = inputs.shape

        # Flatten input: [B, C, H, W] -> [BHW, C]
        flat_input = inputs.permute(0, 2, 3, 1).contiguous()
        flat_input = flat_input.view(-1, self.embedding_dim)

        # Calculate distances - fixed dimensions
        distances = (torch.sum(flat_input**2, dim=1, keepdim=True) +
                    torch.sum(self.embed**2, dim=1) -
                    2 * torch.matmul(flat_input, self.embed.t()))

        # Encoding
        encoding_indices = torch.argmin(distances, dim=1).unsqueeze(1)
        encodings = torch.zeros(encoding_indices.shape[0], self.num_embeddings,
                               device=inputs.device)
        encodings.scatter_(1, encoding_indices, 1)

        # Quantize
        quantized = torch.matmul(encodings, self.embed)

        # Use EMA to update the embedding vectors
        if self.training:
            # Update cluster sizes
            self.cluster_size.data.mul_(self.decay).add_(
                encodings.sum(0), alpha=1 - self.decay
            )

            # Update embedding averages
            dw = torch.matmul(encodings.t(), flat_input)
            self.embed_avg.data.mul_(self.decay).add_(dw, alpha=1 - self.decay)

            # Normalize embeddings
            n = self.cluster_size.sum()
            cluster_size = (
                (self.cluster_size + self.epsilon) /
                (n + self.num_embeddings * self.epsilon) * n
            )
            embed_normalized = self.embed_avg / cluster_size.unsqueeze(1)
            self.embed.data.copy_(embed_normalized)

        # Loss
        e_latent_loss = F.mse_loss(quantized.detach(), flat_input)
        loss = self.commitment_cost * e_latent_loss

        # Straight-through estimator
        quantized = flat_input + (quantized - flat_input).detach()

        # Reshape back
        quantized = quantized.view(input_shape[0], input_shape[2],
                                   input_shape[3], self.embedding_dim)
        quantized = quantized.permute(0, 3, 1, 2).contiguous()

        # Perplexity
        avg_probs = torch.mean(encodings, dim=0)
        perplexity = torch.exp(-torch.sum(avg_probs * torch.log(avg_probs + 1e-10)))

        return quantized, loss, perplexity, encoding_indices

class ResidualBlock(nn.Module):
    """Residual block for encoder/decoder"""

    def __init__(self, in_channels, num_hiddens, num_residual_hiddens):
        super().__init__()
        self.block = nn.Sequential(
            nn.ReLU(),
            nn.Conv2d(in_channels, num_residual_hiddens, 3, padding=1, bias=False),
            nn.BatchNorm2d(num_residual_hiddens),
            nn.ReLU(),
            nn.Conv2d(num_residual_hiddens, num_hiddens, 1, bias=False),
            nn.BatchNorm2d(num_hiddens)
        )

    def forward(self, x):
        return x + self.block(x)

class ResidualStack(nn.Module):
    """Stack of residual blocks"""

    def __init__(self, in_channels, num_hiddens, num_residual_layers, num_residual_hiddens):
        super().__init__()
        self.layers = nn.ModuleList([
            ResidualBlock(in_channels, num_hiddens, num_residual_hiddens)
            for _ in range(num_residual_layers)
        ])

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return F.relu(x)

class Encoder(nn.Module):
    """Encoder network"""

    def __init__(self, in_channels, num_hiddens, num_residual_layers, num_residual_hiddens):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels, num_hiddens // 2, 4, stride=2, padding=1)
        self.conv2 = nn.Conv2d(num_hiddens // 2, num_hiddens, 4, stride=2, padding=1)
        self.conv3 = nn.Conv2d(num_hiddens, num_hiddens, 3, padding=1)
        self.residual_stack = ResidualStack(
            num_hiddens, num_hiddens, num_residual_layers, num_residual_hiddens
        )

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.conv3(x)
        return self.residual_stack(x)

class Decoder(nn.Module):
    """Decoder network"""

    def __init__(self, embedding_dim, num_hiddens, num_residual_layers, num_residual_hiddens):
        super().__init__()
        self.conv1 = nn.Conv2d(embedding_dim, num_hiddens, 3, padding=1)
        self.residual_stack = ResidualStack(
            num_hiddens, num_hiddens, num_residual_layers, num_residual_hiddens
        )
        self.conv_trans1 = nn.ConvTranspose2d(num_hiddens, num_hiddens // 2,
                                              4, stride=2, padding=1)
        self.conv_trans2 = nn.ConvTranspose2d(num_hiddens // 2, 3,
                                              4, stride=2, padding=1)

    def forward(self, x):
        x = self.conv1(x)
        x = self.residual_stack(x)
        x = F.relu(self.conv_trans1(x))
        return torch.tanh(self.conv_trans2(x))

# ============================================================================
# FIX 2: Debug the spatial size issue
# ============================================================================
# Add this function to check architecture:
def verify_architecture():
    """Verify the VQ-VAE produces correct spatial sizes"""
    print("\nVerifying architecture...")
    model = VQVAE(config).to(config.device)

    # Test with dummy input
    dummy_input = torch.randn(1, 3, 64, 64).to(config.device)

    with torch.no_grad():
        # Check encoder output
        z = model.encoder(dummy_input)
        print(f"After encoder: {z.shape}")

        z = model.pre_vq_conv(z)
        print(f"After pre_vq_conv: {z.shape}")

        # Check VQ
        quantized, _, _, codes = model.vq(z)
        print(f"After VQ: {quantized.shape}")
        print(f"Codes shape: {codes.shape}")

        # Check decoder
        recon = model.decoder(quantized)
        print(f"After decoder: {recon.shape}")

        # Calculate spatial size
        spatial_size = z.shape[2]
        print(f"\nExpected latent spatial size: {spatial_size}x{spatial_size}")

    return model, spatial_size

class VQVAE(nn.Module):
    """Complete VQ-VAE model with EMA"""

    def __init__(self, config, use_ema=True):
        super().__init__()
        self.encoder = Encoder(3, config.num_hiddens, config.num_residual_layers,
                              config.num_residual_hiddens)
        self.pre_vq_conv = nn.Conv2d(config.num_hiddens, config.embedding_dim, 1)

        # Choose VQ method
        if use_ema:
            self.vq = VectorQuantizerEMA(
                config.num_embeddings,
                config.embedding_dim,
                config.commitment_cost,
                decay=config.decay
            )
        else:
            self.vq = VectorQuantizerEMA(
                config.num_embeddings,
                config.embedding_dim,
                config.commitment_cost
            )

        self.decoder = Decoder(config.embedding_dim, config.num_hiddens,
                              config.num_residual_layers, config.num_residual_hiddens)

    def forward(self, x):
        z = self.encoder(x)
        z = self.pre_vq_conv(z)
        quantized, vq_loss, perplexity, encoding_indices = self.vq(z)
        x_recon = self.decoder(quantized)
        return x_recon, vq_loss, perplexity, encoding_indices

    def encode(self, x):
        """Encode to discrete codes"""
        z = self.encoder(x)
        z = self.pre_vq_conv(z)
        _, _, _, encoding_indices = self.vq(z)
        # Reshape to [B, H*W]
        B = x.shape[0]
        encoding_indices = encoding_indices.view(B, -1)
        return encoding_indices

    def decode_codes(self, encoding_indices, spatial_size):
        """Decode from discrete codes"""
        # Reshape codes to spatial layout
        codes = encoding_indices.view(-1, spatial_size, spatial_size)

        # Get quantized vectors - FIXED for both versions
        if hasattr(self.vq, 'embed'):  # EMA version
            quantized = F.embedding(codes, self.vq.embed)  # Remove .t()
        else:  # Original version
            quantized = self.vq.embeddings(codes)

        quantized = quantized.permute(0, 3, 1, 2).contiguous()

        # Decode
        return self.decoder(quantized)

# ============================================================================
# PixelCNN Prior
# ============================================================================
class MaskedConv2d(nn.Conv2d):
    """Masked convolution for autoregressive modeling"""

    def __init__(self, mask_type, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.register_buffer('mask', torch.zeros_like(self.weight))
        self.create_mask(mask_type)

    def create_mask(self, mask_type):
        k = self.kernel_size[0]
        self.mask[:, :, :k//2, :] = 1
        self.mask[:, :, k//2, :k//2] = 1
        if mask_type == 'B':
            self.mask[:, :, k//2, k//2] = 1

    def forward(self, x):
        self.weight.data *= self.mask
        return super().forward(x)

class PixelCNNResidualBlock(nn.Module):
    """Residual block for PixelCNN"""

    def __init__(self, h):
        super().__init__()
        self.conv = nn.Sequential(
            nn.ReLU(),
            MaskedConv2d('B', h, h, 1),
            nn.BatchNorm2d(h),
            nn.ReLU(),
            MaskedConv2d('B', h, h, 1),
            nn.BatchNorm2d(h)
        )

    def forward(self, x):
        return x + self.conv(x)

class PixelCNN(nn.Module):
    """PixelCNN prior for discrete codes with stability improvements"""

    def __init__(self, num_embeddings, num_layers=12, hidden_dim=64):
        super().__init__()
        self.num_embeddings = num_embeddings

        # Input layer
        self.input_conv = MaskedConv2d('A', num_embeddings, hidden_dim, 7, padding=3)

        # Residual blocks
        self.residual_blocks = nn.ModuleList([
            PixelCNNResidualBlock(hidden_dim) for _ in range(num_layers)
        ])

        # Output layers
        self.output = nn.Sequential(
            nn.ReLU(),
            MaskedConv2d('B', hidden_dim, hidden_dim, 1),
            nn.BatchNorm2d(hidden_dim),
            nn.ReLU(),
            nn.Conv2d(hidden_dim, num_embeddings, 1)
        )

        # Initialize weights properly
        self._init_weights()

    def _init_weights(self):
        """Initialize weights with smaller values for stability"""
        for m in self.modules():
            if isinstance(m, (nn.Conv2d, MaskedConv2d)):
                nn.init.xavier_uniform_(m.weight, gain=0.1)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

    def forward(self, x):
        # x: [B, H, W] with values in [0, num_embeddings)
        # Convert to one-hot: [B, num_embeddings, H, W]
        x_onehot = F.one_hot(x, self.num_embeddings).float()
        x_onehot = x_onehot.permute(0, 3, 1, 2).contiguous()

        x = self.input_conv(x_onehot)
        for block in self.residual_blocks:
            x = block(x)
        return self.output(x)

    @torch.no_grad()
    def sample(self, batch_size, spatial_size, device, temperature=1.0):
        """Generate samples autoregressively"""
        samples = torch.zeros(batch_size, spatial_size, spatial_size,
                            dtype=torch.long, device=device)

        for i in range(spatial_size):
            for j in range(spatial_size):
                logits = self(samples)
                probs = F.softmax(logits[:, :, i, j] / temperature, dim=1)
                samples[:, i, j] = torch.multinomial(probs, 1).squeeze(-1)

        return samples

# ============================================================================
# Metrics
# ============================================================================
def calculate_mse(original, reconstructed):
    """Calculate Mean Squared Error"""
    mse = F.mse_loss(reconstructed, original)
    return mse.item()

def calculate_psnr(original, reconstructed, max_val=2.0):
    """Calculate Peak Signal-to-Noise Ratio"""
    mse = F.mse_loss(reconstructed, original)
    psnr = 10 * torch.log10(max_val**2 / mse)
    return psnr.item()

def calculate_ssim(original, reconstructed):
    """Calculate Structural Similarity Index"""
    # Convert to numpy and normalize to [0, 1]
    orig_np = (original.detach().cpu().numpy() + 1) / 2
    recon_np = (reconstructed.detach().cpu().numpy() + 1) / 2

    ssim_scores = []
    for i in range(orig_np.shape[0]):
        score = ssim(orig_np[i].transpose(1, 2, 0),
                    recon_np[i].transpose(1, 2, 0),
                    multichannel=True, data_range=1.0, channel_axis=2)
        ssim_scores.append(score)

    return np.mean(ssim_scores)

def calculate_fid(real_features, fake_features):
    """Calculate Fréchet Inception Distance"""
    mu1, sigma1 = real_features.mean(axis=0), np.cov(real_features, rowvar=False)
    mu2, sigma2 = fake_features.mean(axis=0), np.cov(fake_features, rowvar=False)

    ssdiff = np.sum((mu1 - mu2)**2)
    covmean = linalg.sqrtm(sigma1.dot(sigma2))

    if np.iscomplexobj(covmean):
        covmean = covmean.real

    fid = ssdiff + np.trace(sigma1 + sigma2 - 2*covmean)
    return fid

def get_inception_features(images, model, device):
    """Extract features using a pretrained model"""
    from torchvision.models import inception_v3

    if model is None:
        model = inception_v3(pretrained=True, transform_input=False)
        model.fc = nn.Identity()
        model = model.to(device)
        model.eval()

    with torch.no_grad():
        # Resize images to 299x299 for Inception
        images_resized = F.interpolate(images, size=(299, 299), mode='bilinear')
        features = model(images_resized)

    return features.cpu().numpy(), model

# ============================================================================
# Training Functions
# ============================================================================
class Trainer:
    """Trainer class for VQ-VAE"""

    def __init__(self, model, config):
        self.model = model
        self.config = config
        self.optimizer = torch.optim.Adam(model.parameters(), lr=config.learning_rate_vqvae)
        self.history = {
            'recon_loss': [], 'vq_loss': [], 'total_loss': [],
            'perplexity': [], 'mse': [], 'psnr': [], 'ssim': []
        }
        self.start_epoch = 0

    def save_checkpoint(self, epoch, filepath):
        """Save training checkpoint"""
        checkpoint = {
            'epoch': epoch,
            'model_state_dict': self.model.state_dict(),
            'optimizer_state_dict': self.optimizer.state_dict(),
            'history': self.history
        }
        torch.save(checkpoint, filepath)
        print(f"Checkpoint saved: {filepath}")

    def load_checkpoint(self, filepath):
        """Load training checkpoint"""
        if os.path.exists(filepath):
            try:
                # First try with weights_only=True (safe)
                checkpoint = torch.load(filepath, map_location=self.config.device, weights_only=True)
            except Exception as e:
                print(f"Safe loading failed: {e}")
                print("Attempting unsafe load (only use if you trust the checkpoint source)...")
                # Fallback to unsafe load
                checkpoint = torch.load(filepath, map_location=self.config.device, weights_only=False)

            self.model.load_state_dict(checkpoint['model_state_dict'])
            self.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
            self.history = checkpoint['history']
            self.start_epoch = checkpoint['epoch'] + 1
            print(f"Checkpoint loaded: {filepath}, resuming from epoch {self.start_epoch}")
            return True
        return False

    def train_epoch(self, dataloader):
        """Train for one epoch"""
        self.model.train()
        epoch_recon_loss = 0
        epoch_vq_loss = 0
        epoch_perplexity = 0

        pbar = tqdm(dataloader, desc="Training")
        for batch in pbar:
            batch = batch.to(self.config.device)

            # Forward pass
            recon, vq_loss, perplexity, _ = self.model(batch)
            recon_loss = F.mse_loss(recon, batch)
            loss = recon_loss + vq_loss

            # Backward pass
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()

            # Track metrics
            epoch_recon_loss += recon_loss.item()
            epoch_vq_loss += vq_loss.item()
            epoch_perplexity += perplexity.item()

            pbar.set_postfix({
                'recon_loss': recon_loss.item(),
                'vq_loss': vq_loss.item(),
                'perplexity': perplexity.item()
            })

        n_batches = len(dataloader)
        return {
            'recon_loss': epoch_recon_loss / n_batches,
            'vq_loss': epoch_vq_loss / n_batches,
            'perplexity': epoch_perplexity / n_batches
        }

    @torch.no_grad()
    def evaluate(self, dataloader):
        """Evaluate model"""
        self.model.eval()
        all_original = []
        all_recon = []

        for batch in dataloader:
            batch = batch.to(self.config.device)
            recon, _, _, _ = self.model(batch)
            all_original.append(batch)
            all_recon.append(recon)

        original = torch.cat(all_original, dim=0)
        reconstructed = torch.cat(all_recon, dim=0)

        mse = calculate_mse(original, reconstructed)
        psnr = calculate_psnr(original, reconstructed)
        ssim_score = calculate_ssim(original[:64], reconstructed[:64])  # Sample for speed

        return {'mse': mse, 'psnr': psnr, 'ssim': ssim_score}

    def get_codebook_utilization(self, dataloader):
        """Calculate codebook utilization percentage"""
        self.model.eval()
        all_codes = []

        with torch.no_grad():
            for batch in tqdm(dataloader, desc="Calculating codebook utilization"):
                batch = batch.to(self.config.device)
                codes = self.model.encode(batch)
                all_codes.append(codes.cpu())

        all_codes = torch.cat(all_codes, dim=0).numpy().flatten()
        unique_codes = len(np.unique(all_codes))
        utilization_rate = unique_codes / self.config.num_embeddings

        return utilization_rate, unique_codes

    def train_phase1_codebook(self, train_loader, val_loader, target_utilization=0.5):
        """Phase 1: Train until codebook utilization reaches target"""
        print(f"Starting Phase 1: Training until codebook utilization >= {target_utilization*100}%")

        for epoch in range(self.start_epoch, self.config.num_epochs_vqvae):
            print(f"\nEpoch {epoch+1}/{self.config.num_epochs_vqvae}")

            # Train
            train_metrics = self.train_epoch(train_loader)

            # Evaluate
            val_metrics = self.evaluate(val_loader)

            # Check codebook utilization
            utilization_rate, unique_codes = self.get_codebook_utilization(train_loader)

            # Combine metrics
            metrics = {**train_metrics, **val_metrics}
            metrics['utilization_rate'] = utilization_rate
            metrics['unique_codes'] = unique_codes

            for key, value in metrics.items():
                if key in self.history:
                    self.history[key].append(value)

            # Print metrics
            print(f"Recon Loss: {metrics['recon_loss']:.4f}, "
                  f"VQ Loss: {metrics['vq_loss']:.4f}, "
                  f"Perplexity: {metrics['perplexity']:.2f}")
            print(f"MSE: {metrics['mse']:.4f}, "
                  f"PSNR: {metrics['psnr']:.2f}, "
                  f"SSIM: {metrics['ssim']:.4f}")
            print(f"Codebook Utilization: {unique_codes}/{self.config.num_embeddings} "
                  f"({utilization_rate*100:.2f}%)")

            # Save checkpoint every 5 epochs
            if (epoch + 1) % 5 == 0:
                checkpoint_path = os.path.join(
                    self.config.checkpoint_dir,
                    f'vqvae_phase1_epoch{epoch+1}.pt'
                )
                self.save_checkpoint(epoch, checkpoint_path)

            # Check if we've reached target utilization
            if utilization_rate >= target_utilization:
                print(f"\n🎉 Target codebook utilization reached! "
                      f"{utilization_rate*100:.2f}% >= {target_utilization*100}%")
                checkpoint_path = os.path.join(
                    self.config.checkpoint_dir,
                    f'vqvae_phase1_complete_epoch{epoch+1}.pt'
                )
                self.save_checkpoint(epoch, checkpoint_path)
                return epoch + 1, metrics

        print(f"\n⚠️  Target utilization not reached after {self.config.num_epochs_vqvae} epochs")
        return self.config.num_epochs_vqvae, metrics

    def train_phase2_final(self, train_loader, val_loader, start_epoch):
        """Phase 2: Continue training for final quality"""
        print(f"\nStarting Phase 2: Final training from epoch {start_epoch}")

        for epoch in range(start_epoch, self.config.num_epochs_vqvae):
            print(f"\nEpoch {epoch+1}/{self.config.num_epochs_vqvae}")

            # Train
            train_metrics = self.train_epoch(train_loader)

            # Evaluate
            val_metrics = self.evaluate(val_loader)

            # Combine metrics
            metrics = {**train_metrics, **val_metrics}
            for key, value in metrics.items():
                self.history[key].append(value)

            # Print metrics
            print(f"Recon Loss: {metrics['recon_loss']:.4f}, "
                  f"VQ Loss: {metrics['vq_loss']:.4f}, "
                  f"Perplexity: {metrics['perplexity']:.2f}")
            print(f"MSE: {metrics['mse']:.4f}, "
                  f"PSNR: {metrics['psnr']:.2f}, "
                  f"SSIM: {metrics['ssim']:.4f}")

            # Save checkpoint every 5 epochs
            if (epoch + 1) % 5 == 0:
                checkpoint_path = os.path.join(
                    self.config.checkpoint_dir,
                    f'vqvae_phase2_epoch{epoch+1}.pt'
                )
                self.save_checkpoint(epoch, checkpoint_path)

        # Save final model
        final_path = os.path.join(self.config.checkpoint_dir, 'vqvae_final.pt')
        self.save_checkpoint(self.config.num_epochs_vqvae - 1, final_path)

        return self.history

# ============================================================================
# Enhanced Analysis Functions (From Code 2)
# ============================================================================
def analyze_codebook_tsne(model, dataloader, device, save_path, num_samples=1000):
    """Enhanced t-SNE analysis of codebook usage"""
    model.eval()
    all_codes = []

    print("Extracting codes for t-SNE analysis...")
    with torch.no_grad():
        for batch in tqdm(dataloader):
            batch = batch.to(device)
            codes = model.encode(batch)
            all_codes.append(codes.cpu())
            if len(all_codes) * config.batch_size >= num_samples:
                break

    all_codes = torch.cat(all_codes, dim=0).numpy()[:num_samples]

    # Apply t-SNE if we have more than 2 dimensions
    if all_codes.shape[1] > 2:
        print("Applying t-SNE...")
        tsne = TSNE(n_components=2, random_state=42, perplexity=30)
        codes_2d = tsne.fit_transform(all_codes)
    else:
        codes_2d = all_codes

    # Plot t-SNE
    plt.figure(figsize=(10, 8))
    plt.scatter(codes_2d[:, 0], codes_2d[:, 1], alpha=0.6, s=10)
    plt.title('t-SNE Visualization of Latent Codes')
    plt.xlabel('t-SNE Component 1')
    plt.ylabel('t-SNE Component 2')
    plt.grid(True, alpha=0.3)
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"t-SNE visualization saved to {save_path}")

    return codes_2d

def analyze_codebook_clustering(model, dataloader, device, save_path, n_clusters=10, num_samples=1000):
    """Enhanced clustering analysis"""
    model.eval()
    all_codes = []
    all_images = []

    print("Extracting codes and images for clustering...")
    with torch.no_grad():
        for batch in tqdm(dataloader):
            batch = batch.to(device)
            codes = model.encode(batch)
            all_codes.append(codes.cpu())
            all_images.append(batch.cpu())
            if len(all_codes) * config.batch_size >= num_samples:
                break

    all_codes = torch.cat(all_codes, dim=0).numpy()[:num_samples]
    all_images = torch.cat(all_images, dim=0)[:num_samples]

    # Apply t-SNE for visualization
    if all_codes.shape[1] > 2:
        tsne = TSNE(n_components=2, random_state=42)
        codes_2d = tsne.fit_transform(all_codes)
    else:
        codes_2d = all_codes

    # K-means clustering
    print(f"Performing K-means clustering with {n_clusters} clusters...")
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    cluster_labels = kmeans.fit_predict(all_codes)

    # Plot clustering results
    plt.figure(figsize=(12, 5))

    plt.subplot(1, 2, 1)
    scatter = plt.scatter(codes_2d[:, 0], codes_2d[:, 1], c=cluster_labels,
                         alpha=0.6, cmap='tab10', s=20)
    plt.title('K-means Clustering of Latent Codes')
    plt.xlabel('Component 1')
    plt.ylabel('Component 2')
    plt.colorbar(scatter)
    plt.grid(True, alpha=0.3)

    # Show cluster sizes
    plt.subplot(1, 2, 2)
    cluster_sizes = np.bincount(cluster_labels, minlength=n_clusters)
    plt.bar(range(n_clusters), cluster_sizes)
    plt.title('Cluster Sizes Distribution')
    plt.xlabel('Cluster ID')
    plt.ylabel('Number of Samples')
    plt.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"Clustering analysis saved to {save_path}")

    return cluster_labels, kmeans, codes_2d

def enhanced_codebook_analysis(model, dataloader, device, save_dir):
    """Comprehensive codebook analysis combining both approaches"""
    os.makedirs(save_dir, exist_ok=True)

    # 1. Basic codebook usage (from Code 1)
    codebook_stats = visualize_codebook_usage(
        model, dataloader, device,
        os.path.join(save_dir, 'codebook_usage.png')
    )

    # 2. t-SNE analysis (from Code 2)
    codes_2d = analyze_codebook_tsne(
        model, dataloader, device,
        os.path.join(save_dir, 'tsne_codes.png')
    )

    # 3. Clustering analysis (from Code 2)
    cluster_labels, kmeans, codes_2d_cluster = analyze_codebook_clustering(
        model, dataloader, device,
        os.path.join(save_dir, 'clustering_analysis.png')
    )

    # 4. Combined analysis plot
    plt.figure(figsize=(15, 5))

    # Code usage histogram
    plt.subplot(1, 3, 1)
    code_counts = codebook_stats['code_counts']
    plt.hist(code_counts[code_counts > 0], bins=50, edgecolor='black', alpha=0.7)
    plt.title(f'Code Usage Distribution\n(Active: {codebook_stats["active_codes"]}/{codebook_stats["total_codes"]})')
    plt.xlabel('Usage Count')
    plt.ylabel('Frequency')
    plt.grid(True, alpha=0.3)

    # t-SNE visualization
    plt.subplot(1, 3, 2)
    plt.scatter(codes_2d[:, 0], codes_2d[:, 1], alpha=0.6, s=10)
    plt.title('Latent Codes t-SNE')
    plt.xlabel('t-SNE 1')
    plt.ylabel('t-SNE 2')
    plt.grid(True, alpha=0.3)

    # Clustering visualization
    plt.subplot(1, 3, 3)
    scatter = plt.scatter(codes_2d_cluster[:, 0], codes_2d_cluster[:, 1],
                         c=cluster_labels, alpha=0.6, cmap='tab10', s=10)
    plt.title('K-means Clustering')
    plt.xlabel('Component 1')
    plt.ylabel('Component 2')
    plt.colorbar(scatter)
    plt.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig(os.path.join(save_dir, 'comprehensive_codebook_analysis.png'),
                dpi=300, bbox_inches='tight')
    plt.close()

    print(f"Comprehensive codebook analysis saved to {save_dir}")

    return {
        'codebook_stats': codebook_stats,
        'clustering': {
            'labels': cluster_labels,
            'kmeans': kmeans,
            'codes_2d': codes_2d_cluster
        },
        'tsne': codes_2d
    }

# ============================================================================
# Visualization Functions (Keep your existing ones but ensure they're correct)
# ============================================================================
def plot_training_history(history, save_path):
    """Plot training history"""
    fig, axes = plt.subplots(2, 4, figsize=(20, 10))

    # Plot losses
    axes[0, 0].plot(history['recon_loss'], label='Reconstruction Loss')
    axes[0, 0].set_xlabel('Epoch')
    axes[0, 0].set_ylabel('Loss')
    axes[0, 0].set_title('Reconstruction Loss')
    axes[0, 0].legend()
    axes[0, 0].grid(True)

    axes[0, 1].plot(history['vq_loss'], label='VQ Loss', color='orange')
    axes[0, 1].set_xlabel('Epoch')
    axes[0, 1].set_ylabel('Loss')
    axes[0, 1].set_title('VQ Loss')
    axes[0, 1].legend()
    axes[0, 1].grid(True)

    axes[0, 2].plot(history['perplexity'], label='Perplexity', color='green')
    axes[0, 2].set_xlabel('Epoch')
    axes[0, 2].set_ylabel('Perplexity')
    axes[0, 2].set_title('Codebook Perplexity')
    axes[0, 2].legend()
    axes[0, 2].grid(True)

    # Plot utilization rate if available
    if 'utilization_rate' in history:
        axes[0, 3].plot([x * 100 for x in history['utilization_rate']], label='Utilization Rate', color='purple')
        axes[0, 3].set_xlabel('Epoch')
        axes[0, 3].set_ylabel('Utilization (%)')
        axes[0, 3].set_title('Codebook Utilization Rate')
        axes[0, 3].legend()
        axes[0, 3].grid(True)

    # Plot metrics
    axes[1, 0].plot(history['mse'], label='MSE', color='red')
    axes[1, 0].set_xlabel('Epoch')
    axes[1, 0].set_ylabel('MSE')
    axes[1, 0].set_title('Mean Squared Error')
    axes[1, 0].legend()
    axes[1, 0].grid(True)

    axes[1, 1].plot(history['psnr'], label='PSNR', color='purple')
    axes[1, 1].set_xlabel('Epoch')
    axes[1, 1].set_ylabel('PSNR (dB)')
    axes[1, 1].set_title('Peak Signal-to-Noise Ratio')
    axes[1, 1].legend()
    axes[1, 1].grid(True)

    axes[1, 2].plot(history['ssim'], label='SSIM', color='brown')
    axes[1, 2].set_xlabel('Epoch')
    axes[1, 2].set_ylabel('SSIM')
    axes[1, 2].set_title('Structural Similarity Index')
    axes[1, 2].legend()
    axes[1, 2].grid(True)

    # Hide unused subplot
    axes[1, 3].axis('off')

    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"Training history plot saved to {save_path}")

def plot_reconstructions(model, dataloader, device, save_path, num_images=8):
    """Plot original vs reconstructed images"""
    model.eval()

    # Get a batch
    images = next(iter(dataloader))[:num_images].to(device)

    with torch.no_grad():
        reconstructions, _, _, _ = model(images)

    # Denormalize
    images = (images + 1) / 2
    reconstructions = (reconstructions + 1) / 2

    # Create grid
    comparison = torch.cat([images, reconstructions])
    grid = make_grid(comparison, nrow=num_images, padding=2)

    # Plot
    plt.figure(figsize=(16, 4))
    plt.imshow(grid.permute(1, 2, 0).cpu().numpy())
    plt.axis('off')
    plt.title(f'Top: Original, Bottom: Reconstructed')
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"Reconstruction comparison saved to {save_path}")

def plot_generated_samples(samples, save_path, title="Generated Samples"):
    """Plot generated emoji samples"""
    # Denormalize
    samples = (samples + 1) / 2

    # Create grid
    grid = make_grid(samples, nrow=8, padding=2)

    # Plot
    plt.figure(figsize=(12, 12))
    plt.imshow(grid.permute(1, 2, 0).cpu().numpy())
    plt.axis('off')
    plt.title(title)
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"Generated samples saved to {save_path}")

def visualize_codebook_usage(model, dataloader, device, save_path):
    """Analyze and visualize codebook usage"""
    model.eval()
    all_codes = []

    print("Analyzing codebook usage...")
    with torch.no_grad():
        for batch in tqdm(dataloader):
            batch = batch.to(device)
            codes = model.encode(batch)
            all_codes.append(codes.cpu())

    all_codes = torch.cat(all_codes, dim=0).numpy().flatten()

    # Plot histogram
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))

    # Histogram
    axes[0].hist(all_codes, bins=config.num_embeddings, edgecolor='black')
    axes[0].set_xlabel('Code Index')
    axes[0].set_ylabel('Frequency')
    axes[0].set_title('Codebook Usage Distribution')
    axes[0].grid(True, alpha=0.3)

    # Usage statistics
    unique_codes = len(np.unique(all_codes))
    usage_rate = unique_codes / config.num_embeddings * 100

    code_counts = np.bincount(all_codes, minlength=config.num_embeddings)
    axes[1].bar(range(len(code_counts)), sorted(code_counts, reverse=True))
    axes[1].set_xlabel('Code Rank')
    axes[1].set_ylabel('Usage Count')
    axes[1].set_title(f'Sorted Code Usage (Active: {unique_codes}/{config.num_embeddings} = {usage_rate:.1f}%)')
    axes[1].grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"Codebook usage plot saved to {save_path}")

    return {
        'total_codes': config.num_embeddings,
        'active_codes': unique_codes,
        'usage_rate': usage_rate,
        'code_counts': code_counts
    }

def visualize_latent_space(model, dataloader, device, save_path):
    """Visualize latent space using t-SNE"""
    model.eval()
    all_latents = []
    all_images = []

    print("Extracting latent representations...")
    with torch.no_grad():
        for batch in tqdm(dataloader):
            batch = batch.to(device)
            z = model.encoder(batch)
            z = model.pre_vq_conv(z)
            # Pool spatial dimensions
            z_pooled = z.mean(dim=[2, 3])
            all_latents.append(z_pooled.cpu())
            all_images.append(batch.cpu())

    latents = torch.cat(all_latents, dim=0).numpy()
    images = torch.cat(all_images, dim=0)

    # Apply t-SNE
    print("Applying t-SNE...")
    tsne = TSNE(n_components=2, random_state=42, perplexity=30)
    latents_2d = tsne.fit_transform(latents[:1000])  # Use subset for speed

    # Plot
    fig, ax = plt.subplots(1, 1, figsize=(10, 10))
    scatter = ax.scatter(latents_2d[:, 0], latents_2d[:, 1],
                        c=range(len(latents_2d)), cmap='viridis',
                        alpha=0.6, s=20)
    ax.set_title('t-SNE Visualization of Latent Space')
    ax.set_xlabel('t-SNE Component 1')
    ax.set_ylabel('t-SNE Component 2')
    plt.colorbar(scatter, ax=ax, label='Sample Index')

    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"Latent space visualization saved to {save_path}")

    return latents_2d

def visualize_latent_interpolation(model, dataloader, device, save_path, num_steps=10):
    """Interpolate between two emojis in latent space"""
    model.eval()

    # Get two random images
    images = next(iter(dataloader))[:2].to(device)

    with torch.no_grad():
        # Encode to latent
        z1 = model.encoder(images[0:1])
        z1 = model.pre_vq_conv(z1)

        z2 = model.encoder(images[1:2])
        z2 = model.pre_vq_conv(z2)

        # Interpolate
        interpolations = []
        for alpha in np.linspace(0, 1, num_steps):
            z_interp = (1 - alpha) * z1 + alpha * z2

            # Quantize
            quantized, _, _, _ = model.vq(z_interp)

            # Decode
            recon = model.decoder(quantized)
            interpolations.append(recon)

        interpolations = torch.cat(interpolations, dim=0)

    # Denormalize
    interpolations = (interpolations + 1) / 2
    images = (images + 1) / 2

    # Create grid
    grid = make_grid(interpolations, nrow=num_steps, padding=2)

    # Plot
    fig, axes = plt.subplots(2, 1, figsize=(15, 6))

    # Show start and end
    axes[0].imshow(make_grid(images, nrow=2, padding=2).permute(1, 2, 0).cpu().numpy())
    axes[0].set_title('Start and End Emojis')
    axes[0].axis('off')

    # Show interpolation
    axes[1].imshow(grid.permute(1, 2, 0).cpu().numpy())
    axes[1].set_title(f'Latent Space Interpolation ({num_steps} steps)')
    axes[1].axis('off')

    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"Latent interpolation saved to {save_path}")

# ============================================================================
# Prior Trainer (Keep your existing PriorTrainer)
# ============================================================================
class PriorTrainer:
    """Trainer for PixelCNN prior with stability improvements"""

    def __init__(self, prior, vqvae, config):
        self.prior = prior
        self.vqvae = vqvae
        self.config = config
        self.optimizer = torch.optim.Adam(prior.parameters(), lr=config.learning_rate_prior)
        self.history = {'loss': []}
        self.start_epoch = 0

    def save_checkpoint(self, epoch, filepath):
        """Save training checkpoint"""
        checkpoint = {
            'epoch': epoch,
            'model_state_dict': self.prior.state_dict(),
            'optimizer_state_dict': self.optimizer.state_dict(),
            'history': self.history
        }
        torch.save(checkpoint, filepath)
        print(f"Checkpoint saved: {filepath}")

    def load_checkpoint(self, filepath):
        """Load training checkpoint"""
        if os.path.exists(filepath):
            try:
                checkpoint = torch.load(filepath, map_location=self.config.device, weights_only=True)
            except Exception as e:
                print(f"Safe loading failed: {e}")
                print("Attempting unsafe load...")
                checkpoint = torch.load(filepath, map_location=self.config.device, weights_only=False)

            self.prior.load_state_dict(checkpoint['model_state_dict'])
            self.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
            self.history = checkpoint['history']
            self.start_epoch = checkpoint['epoch'] + 1
            print(f"Checkpoint loaded: {filepath}, resuming from epoch {self.start_epoch}")
            return True
        return False

    def train_epoch(self, dataloader):
        """Train for one epoch with gradient clipping"""
        self.prior.train()
        self.vqvae.eval()
        epoch_loss = 0
        num_valid_batches = 0

        pbar = tqdm(dataloader, desc="Training Prior")
        for batch in pbar:
            batch = batch.to(self.config.device)

            # Get discrete codes
            with torch.no_grad():
                codes = self.vqvae.encode(batch)
                spatial_size = int(np.sqrt(codes.shape[1]))
                codes = codes.view(-1, spatial_size, spatial_size)

            # Forward pass
            logits = self.prior(codes)
            loss = F.cross_entropy(logits, codes)

            # Check for NaN
            if torch.isnan(loss):
                print(f"Warning: NaN loss detected, skipping batch")
                continue

            # Backward pass with gradient clipping
            self.optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(self.prior.parameters(), self.config.grad_clip)
            self.optimizer.step()

            epoch_loss += loss.item()
            num_valid_batches += 1
            pbar.set_postfix({'loss': loss.item()})

        if num_valid_batches == 0:
            return float('inf')

        return epoch_loss / num_valid_batches

    def train(self, train_loader):
        """Full training loop"""
        print(f"Starting prior training from epoch {self.start_epoch}")

        for epoch in range(self.start_epoch, self.config.num_epochs_prior):
            print(f"\nEpoch {epoch+1}/{self.config.num_epochs_prior}")

            loss = self.train_epoch(train_loader)

            if loss == float('inf'):
                print("Training failed due to NaN losses. Stopping.")
                break

            self.history['loss'].append(loss)

            print(f"Loss: {loss:.4f}")

            # Save checkpoint every 5 epochs
            if (epoch + 1) % 5 == 0:
                checkpoint_path = os.path.join(
                    self.config.checkpoint_dir,
                    f'prior_epoch{epoch+1}.pt'
                )
                self.save_checkpoint(epoch, checkpoint_path)

        # Save final model
        final_path = os.path.join(self.config.checkpoint_dir, 'prior_final.pt')
        self.save_checkpoint(self.config.num_epochs_prior - 1, final_path)

        return self.history

# ============================================================================
# Main Training and Evaluation Pipeline - ENHANCED
# ============================================================================
def main():
    """Enhanced main function with two-phase training"""

    print("=" * 80)
    print("VQ-VAE Emoji Generation Project - ENHANCED VERSION")
    print("=" * 80)

    # Step 1: Download and prepare dataset
    print("\n" + "=" * 80)
    print("STEP 1: Dataset Preparation")
    print("=" * 80)

    if not os.listdir(config.data_dir):
        download_emoji_dataset()
    else:
        print(f"Dataset already exists in {config.data_dir}")

    # Create dataset and dataloaders
    dataset = EmojiDataset(config.data_dir, config.image_size)
    print(f"Total dataset size: {len(dataset)} images")

    # Split into train and validation
    train_size = int(0.9 * len(dataset))
    val_size = len(dataset) - train_size
    train_dataset, val_dataset = torch.utils.data.random_split(
        dataset, [train_size, val_size]
    )

    train_loader = DataLoader(
        train_dataset, batch_size=config.batch_size,
        shuffle=True, num_workers=config.num_workers, pin_memory=True
    )
    val_loader = DataLoader(
        val_dataset, batch_size=config.batch_size,
        shuffle=False, num_workers=config.num_workers, pin_memory=True
    )

    print(f"Training set: {len(train_dataset)} images")
    print(f"Validation set: {len(val_dataset)} images")

    # Step 2: Architecture Verification
    print("\n" + "=" * 80)
    print("STEP 2: Architecture Verification")
    print("=" * 80)
    model_test, expected_spatial_size = verify_architecture()
    print(f"\nArchitecture verified! Latent spatial size: {expected_spatial_size}x{expected_spatial_size}")
    del model_test  # Free memory

    # Step 3: Two-Phase VQ-VAE Training
    print("\n" + "=" * 80)
    print("STEP 3: Two-Phase VQ-VAE Training")
    print("=" * 80)

    vqvae = VQVAE(config, use_ema=True).to(config.device)
    print(f"VQ-VAE parameters: {sum(p.numel() for p in vqvae.parameters()):,}")

    trainer = Trainer(vqvae, config)

    # Try to load checkpoint
    checkpoint_path = os.path.join(config.checkpoint_dir, 'vqvae_final.pt')
    if trainer.load_checkpoint(checkpoint_path):
        print("Found existing checkpoint, checking codebook utilization...")
        utilization_rate, unique_codes = trainer.get_codebook_utilization(train_loader)
        print(f"Current codebook utilization: {unique_codes}/{config.num_embeddings} ({utilization_rate*100:.2f}%)")

        if utilization_rate >= config.codebook_utilization_threshold:
            print("Codebook utilization already meets threshold! Proceeding to Phase 2 analysis.")
            phase1_complete_epoch = config.num_epochs_vqvae
        else:
            print("Codebook utilization below threshold. Starting Phase 1 training...")
            phase1_complete_epoch, phase1_metrics = trainer.train_phase1_codebook(
                train_loader, val_loader, config.codebook_utilization_threshold
            )
    else:
        print("No checkpoint found, starting Phase 1 training from scratch...")
        phase1_complete_epoch, phase1_metrics = trainer.train_phase1_codebook(
            train_loader, val_loader, config.codebook_utilization_threshold
        )

    # Continue with Phase 2 if we haven't reached the end
    if phase1_complete_epoch < config.num_epochs_vqvae:
        history = trainer.train_phase2_final(train_loader, val_loader, phase1_complete_epoch)
    else:
        history = trainer.history
        print("VQ-VAE training complete!")

    # Plot training history
    plot_training_history(
        history,
        os.path.join(config.results_dir, 'training_history.png')
    )

    # Step 4: Enhanced VQ-VAE Analysis
    print("\n" + "=" * 80)
    print("STEP 4: Enhanced VQ-VAE Analysis")
    print("=" * 80)

    # Plot reconstructions
    plot_reconstructions(
        vqvae, val_loader, config.device,
        os.path.join(config.results_dir, 'reconstructions.png')
    )

    # Enhanced codebook analysis (combining both approaches)
    codebook_analysis_results = enhanced_codebook_analysis(
        vqvae, train_loader, config.device,
        os.path.join(config.results_dir, 'codebook_analysis')
    )

    # Visualize latent space
    visualize_latent_space(
        vqvae, train_loader, config.device,
        os.path.join(config.results_dir, 'latent_space_tsne.png')
    )

    # Latent interpolation
    visualize_latent_interpolation(
        vqvae, val_loader, config.device,
        os.path.join(config.results_dir, 'latent_interpolation.png'),
        num_steps=config.num_interpolation_steps
    )

    # Step 5: Train Prior
    print("\n" + "=" * 80)
    print("STEP 5: Prior Training (PixelCNN)")
    print("=" * 80)

    # Calculate spatial size
    sample_batch = next(iter(train_loader))[:1].to(config.device)
    with torch.no_grad():
        codes = vqvae.encode(sample_batch)
        spatial_size = int(np.sqrt(codes.shape[1]))
    print(f"Latent spatial size: {spatial_size}x{spatial_size}")

    prior = PixelCNN(
        config.num_embeddings,
        config.pixelcnn_layers,
        config.pixelcnn_hidden
    ).to(config.device)
    print(f"PixelCNN parameters: {sum(p.numel() for p in prior.parameters()):,}")

    prior_trainer = PriorTrainer(prior, vqvae, config)

    # Try to load checkpoint
    prior_checkpoint_path = os.path.join(config.checkpoint_dir, 'prior_final.pt')
    if not prior_trainer.load_checkpoint(prior_checkpoint_path):
        print("No prior checkpoint found, training from scratch...")

    # Train
    if prior_trainer.start_epoch < config.num_epochs_prior:
        prior_history = prior_trainer.train(train_loader)

        # Plot prior training history
        plt.figure(figsize=(10, 6))
        plt.plot(prior_history['loss'], label='Cross-Entropy Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.title('Prior Training History')
        plt.legend()
        plt.grid(True)
        plt.savefig(
            os.path.join(config.results_dir, 'prior_training_history.png'),
            dpi=300, bbox_inches='tight'
        )
        plt.close()
    else:
        print("Prior training already complete!")

    # Step 6: Generate Novel Emojis
    print("\n" + "=" * 80)
    print("STEP 6: Novel Emoji Generation")
    print("=" * 80)

    print(f"Generating {config.num_samples} novel emojis...")
    prior.eval()
    vqvae.eval()

    with torch.no_grad():
        # Sample from prior
        sampled_codes = prior.sample(
            config.num_samples, spatial_size, config.device, temperature=1.0
        )

        # Decode to images
        generated_images = vqvae.decode_codes(sampled_codes, spatial_size)

    # Plot generated samples
    plot_generated_samples(
        generated_images,
        os.path.join(config.results_dir, 'generated_emojis.png'),
        title="Generated Novel Emojis"
    )

    # Step 7: Calculate FID
    print("\n" + "=" * 80)
    print("STEP 7: Calculating FID Score")
    print("=" * 80)
    print("Extracting features from real images...")
    real_images = []
    for batch in tqdm(val_loader):
        real_images.append(batch)
        if len(real_images) * config.batch_size >= config.num_samples:
            break
    real_images = torch.cat(real_images, dim=0)[:config.num_samples].to(config.device)

    real_features, inception_model = get_inception_features(real_images, None, config.device)
    print("Extracting features from generated images...")
    fake_features, _ = get_inception_features(generated_images, inception_model, config.device)

    fid_score = calculate_fid(real_features, fake_features)
    print(f"\nFID Score: {fid_score:.2f}")

    # Step 8: Generate Comprehensive Report
    print("\n" + "=" * 80)
    print("STEP 8: Generating Final Report")
    print("=" * 80)

    # Get final utilization rate
    final_utilization_rate, final_unique_codes = trainer.get_codebook_utilization(train_loader)

    report = {
        'dataset_size': len(dataset),
        'train_size': len(train_dataset),
        'val_size': len(val_dataset),
        'vqvae_params': sum(p.numel() for p in vqvae.parameters()),
        'prior_params': sum(p.numel() for p in prior.parameters()),
        'final_metrics': {
            'mse': history['mse'][-1] if history['mse'] else None,
            'psnr': history['psnr'][-1] if history['psnr'] else None,
            'ssim': history['ssim'][-1] if history['ssim'] else None,
            'fid': fid_score,
            'perplexity': history['perplexity'][-1] if history['perplexity'] else None
        },
        'codebook_stats': {
            'total_codes': config.num_embeddings,
            'active_codes': int(final_unique_codes),
            'usage_rate': float(final_utilization_rate * 100),
            'target_utilization': config.codebook_utilization_threshold * 100,
            'utilization_achieved': final_utilization_rate >= config.codebook_utilization_threshold
        },
        'training_info': {
            'phase1_completed_at_epoch': phase1_complete_epoch,
            'total_epochs_trained': len(history['recon_loss']) if history['recon_loss'] else 0,
            'final_recon_loss': history['recon_loss'][-1] if history['recon_loss'] else None,
            'final_vq_loss': history['vq_loss'][-1] if history['vq_loss'] else None
        }
    }

    # Save report
    report_path = os.path.join(config.results_dir, 'final_report.json')
    with open(report_path, 'w') as f:
        json.dump(report, f, indent=2)

    print("\n" + "=" * 80)
    print("FINAL RESULTS")
    print("=" * 80)
    print(f"\nReconstruction Quality:")
    print(f"  MSE: {report['final_metrics']['mse']:.4f}")
    print(f"  PSNR: {report['final_metrics']['psnr']:.2f} dB")
    print(f"  SSIM: {report['final_metrics']['ssim']:.4f}")
    print(f"\nGeneration Quality:")
    print(f"  FID: {report['final_metrics']['fid']:.2f}")
    print(f"\nCodebook Analysis:")
    print(f"  Perplexity: {report['final_metrics']['perplexity']:.2f}")
    print(f"  Active codes: {report['codebook_stats']['active_codes']}/{report['codebook_stats']['total_codes']}")
    print(f"  Usage rate: {report['codebook_stats']['usage_rate']:.2f}%")
    print(f"  Target utilization: {report['codebook_stats']['target_utilization']}%")
    print(f"  Utilization achieved: {'✅ YES' if report['codebook_stats']['utilization_achieved'] else '❌ NO'}")
    print(f"\nTraining Info:")
    print(f"  Phase 1 completed at epoch: {report['training_info']['phase1_completed_at_epoch']}")
    print(f"  Total epochs trained: {report['training_info']['total_epochs_trained']}")
    print(f"\nAll results saved to: {config.results_dir}")
    print(f"Report saved to: {report_path}")

    print("\n" + "=" * 80)
    print("PROJECT COMPLETE!")
    print("=" * 80)

# ============================================================================
# Run the complete pipeline
# ============================================================================
if __name__ == "__main__":
    main()

Using device: cpu
VQ-VAE Emoji Generation Project - ENHANCED VERSION

STEP 1: Dataset Preparation


dataset_infos.json:   0%|          | 0.00/720 [00:00<?, ?B/s]

data/train-00000-of-00001-38cc4fa96c139e(…):   0%|          | 0.00/139M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2749 [00:00<?, ? examples/s]

Downloaded 2749 emojis
Saving images to disk...


...

Calculating codebook utilization: 100%|██████████| 39/39 [00:22<00:00,  1.74it/s]


Recon Loss: 0.0602, VQ Loss: 0.0654, Perplexity: 3.70
MSE: 0.0630, PSNR: 18.03, SSIM: 0.6816
Codebook Utilization: 4/512 (0.78%)

Epoch 19/100


Calculating codebook utilization: 100%|██████████| 39/39 [00:22<00:00,  1.74it/s]


Recon Loss: 0.0639, VQ Loss: 0.0728, Perplexity: 3.66
MSE: 0.0642, PSNR: 17.95, SSIM: 0.6750
Codebook Utilization: 4/512 (0.78%)

Epoch 20/100


Training:  49%|████▊     | 19/39 [00:39<00:43,  2.15s/it, recon_loss=0.0673, vq_loss=0.0793, perplexity=3.71]

In [8]:
ls

checkpoints/  results/  sample_data/


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
import pandas as pd

# Display all PNG images
results_dir = '/content/results/'
image_files = [f for f in os.listdir(results_dir) if f.endswith('.png')]

print("Displaying saved plots:")
for img_file in image_files:
    img_path = os.path.join(results_dir, img_file)
    img = mpimg.imread(img_path)
    plt.imshow(img)
    plt.title(img_file)
    plt.axis('off')
    plt.show()

# Print the contents of experiment_results.csv
csv_path = os.path.join(results_dir, 'experiment_results.csv')

if os.path.exists(csv_path):
    print("\nContents of experiment_results.csv:")
    df = pd.read_csv(csv_path)
    for index, row in df.iterrows():
        print(','.join(map(str, row.tolist())))
else:
    print(f"\n{csv_path} not found.")

In [ ]:
import os
import shutil
from google.colab import drive

# Mount Google Drive
print("Mounting Google Drive...")
drive.mount('/content/drive')
print("Google Drive mounted.")

# Define source and destination paths
source_checkpoints = '/content/checkpoints'
source_results = '/content/results'
destination_base = '/content/drive/MyDrive'
destination_folder_name = config.experiment_name
destination_path = os.path.join(destination_base, destination_folder_name)

# Create the destination folder if it doesn't exist
print(f"Creating destination folder: {destination_path}")
os.makedirs(destination_path, exist_ok=True)
print(f"Destination folder created or already exists.")

# Transfer checkpoints folder
destination_checkpoints = os.path.join(destination_path, os.path.basename(source_checkpoints))
print(f"\nTransferring checkpoints from {source_checkpoints} to {destination_checkpoints}...")
if os.path.exists(source_checkpoints):
    if os.path.exists(destination_checkpoints):
        print(f"Removing existing destination checkpoints folder: {destination_checkpoints}")
        shutil.rmtree(destination_checkpoints)
        print(f"Removed existing destination checkpoints folder: {destination_checkpoints}")
    try:
        shutil.copytree(source_checkpoints, destination_checkpoints)
        print("Checkpoints transfer complete.")
        # Verify contents
        source_files = os.listdir(source_checkpoints)
        destination_files = os.listdir(destination_checkpoints)
        print(f"Source files in checkpoints: {source_files}")
        print(f"Destination files in checkpoints: {destination_files}")
        if set(source_files) == set(destination_files) and len(source_files) > 0:
            print("Checkpoints contents successfully verified.")
        elif len(source_files) == 0:
             print("Source checkpoints folder is empty.")
        else:
             print("Warning: Checkpoints contents may not have been fully copied.")
    except Exception as e:
        print(f"Error during checkpoints transfer: {e}")
else:
    print(f"Source checkpoints folder not found: {source_checkpoints}")


# Transfer results folder
destination_results = os.path.join(destination_path, os.path.basename(source_results))
print(f"\nTransferring results from {source_results} to {destination_results}...")
if os.path.exists(source_results):
    if os.path.exists(destination_results):
        print(f"Removing existing destination results folder: {destination_results}")
        shutil.rmtree(destination_results)
        print(f"Removed existing destination results folder: {destination_results}")
    try:
        shutil.copytree(source_results, destination_results)
        print("Results transfer complete.")
        # Verify contents
        source_files = os.listdir(source_results)
        destination_files = os.listdir(destination_results)
        print(f"Source files in results: {source_files}")
        print(f"Destination files in results: {destination_files}")
        if set(source_files) == set(destination_files) and len(source_files) > 0:
            print("Results contents successfully verified.")
        elif len(source_files) == 0:
             print("Source results folder is empty.")
        else:
             print("Warning: Results contents may not have been fully copied.")
    except Exception as e:
        print(f"Error during results transfer: {e}")
else:
    print(f"Source results folder not found: {source_results}")

print("\nTransfer process finished.")

In [ ]:
ls